This is an example using GulpIO with different python based machine learning libraries. The basic idea here is to illustrate flexibility of GulpIO as a agnostic data loader. It decouples data loading and transforming from network definition hence, you are able to use any DL library of your choice without changing the core data part of your code. 


### Gulp Mnist

Download mnist jpeg version from https://github.com/myleott/mnist_png/blob/master/mnist_png.tar.gz and extract it

In [ ]:
! gulp_image_folder --images_per_chunk 5000 --num_workers 8 --shuffle mnist/training/ mnist/trainingGulpIO/
! gulp_image_folder --images_per_chunk 5000 --num_workers 8 --shuffle mnist/testing/ mnist/testingGulpIO/

### Define GulpIO Loader

Here we define the desired transformations and multi-process GulpIO data loader.

In [ ]:
from gulpio.dataset import GulpImageDataset
from gulpio.loader import DataLoader
from gulpio.transforms import Scale, CenterCrop, Compose, UnitNorm

transforms = Compose([
                      Scale(28), 
                      CenterCrop(28),
                      UnitNorm(),
                    ])

dataset = GulpImageDataset('/home/egolge/mnist2/mnist_png/trainingGulpIO/', transform=transforms)
loader = DataLoader(dataset, batch_size=256, shuffle=True, num_workers=0, drop_last=True)

dataset_val = GulpImageDataset('/home/egolge/mnist2/mnist_png/testingGulpIO/', transform=transforms)
loader_val = DataLoader(dataset_val, batch_size=256, shuffle=True, num_workers=0, drop_last=True)

In [ ]:
# Sound check
for data, label in loader:
    print(data.shape)
    break

### Keras Training

taken from https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py

In [ ]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.utils import to_categorical

In [ ]:
num_classes = len(dataset.classes)
if K.image_data_format() == 'channels_first':
    input_shape = (1, 32, 32)
else:
    input_shape = (32, 32, 1)

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [ ]:
# train model for 1 epoch
for data, label in loader:
    model.fit(data[:,:,:,None], to_categorical(label, num_classes=num_classes), batch_size=256, epochs=1, verbose=0)
    num_batch += 1


In [ ]:
# test model
losses = []
accuracies = []
for data, label in loader_val:
    score = model.evaluate(data[:,:,:,None], to_categorical(label, num_classes=num_classes), verbose=0, batch_size=256)
    losses.append(score[0])
    accuracies.append(score[1])

print('Test loss:', np.mean(losses))
print('Test accuracy:', np.mean(accuracies))

### MxNet Training

We use gluonn which is MxNet's dynamix computation graph wrapper. This example is taken from https://github.com/apache/incubator-mxnet/blob/master/example/svm_mnist/svm_mnist.py

In [ ]:
import mxnet as mx
from mxnet import nd, autograd
from mxnet import gluon
import numpy as np

In [ ]:
ctx = mx.gpu()
num_fc = 512
net = gluon.nn.Sequential()
with net.name_scope():
    net.add(gluon.nn.Conv2D(channels=20, kernel_size=5, activation='relu'))
    net.add(gluon.nn.MaxPool2D(pool_size=2, strides=2))            
    net.add(gluon.nn.Conv2D(channels=50, kernel_size=5, activation='relu'))
    net.add(gluon.nn.MaxPool2D(pool_size=2, strides=2))
    # The Flatten layer collapses all axis, except the first one, into one axis.
    net.add(gluon.nn.Flatten())
    net.add(gluon.nn.Dense(num_fc, activation="relu"))
    net.add(gluon.nn.Dense(len(dataset.classes)))
net.collect_params().initialize(mx.init.Xavier(magnitude=2.24), ctx=ctx)
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': .01})

In [ ]:
def evaluate_accuracy(data_iterator, net):
    acc = mx.metric.Accuracy()
    for i, (data, label) in enumerate(data_iterator):
        # pass numpy to mxnet
        data = mx.nd.array(data[:,None,:,:])
        label = mx.nd.array(label)
        
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        output = net(data)
        predictions = nd.argmax(output, axis=1)
        acc.update(preds=predictions, labels=label)
    return acc.get()[1]

In [ ]:
epochs = 10
smoothing_constant = .01

for e in range(epochs):
    for i, (data, label) in enumerate(loader):
        # pass numpy to mxnet
        data = mx.nd.array(data[:,None,:,:])
        label = mx.nd.array(label)
        
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        with autograd.record():
            output = net(data)
            loss = softmax_cross_entropy(output, label)
        loss.backward()
        trainer.step(data.shape[0])

        curr_loss = nd.mean(loss).asscalar()
        moving_loss = (curr_loss if ((i == 0) and (e == 0)) 
                       else (1 - smoothing_constant) * moving_loss + (smoothing_constant) * curr_loss)
        
    test_accuracy = evaluate_accuracy(loader_val, net)
    train_accuracy = evaluate_accuracy(loader, net)
    print("Epoch %s. Loss: %s, Train_acc %s, Test_acc %s" % (e, moving_loss, train_accuracy, test_accuracy))

### PyTorch

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

model = Net()
model.cuda()

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Train for 1 epoch
model.train()
for batch_idx, (data, target) in enumerate(loader):
    data, target = torch.FloatTensor(data[:, None, :, :]).cuda(), torch.LongTensor(target).cuda()
    data, target = Variable(data), Variable(target)
    optimizer.zero_grad()
    output = model(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % 10 == 0:
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(loader.dataset),
            100. * batch_idx / len(loader), loss.data[0]))

# Eval on test set
model.eval()
test_loss = 0
correct = 0
for data, target in loader_val:
    data, target = torch.FloatTensor(data[:, None, :, :]).cuda(), torch.LongTensor(target).cuda()
    data, target = Variable(data, volatile=True), Variable(target)
    output = model(data)
    test_loss += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
    pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
    correct += pred.eq(target.data.view_as(pred)).cpu().sum()

test_loss /= len(loader_val.dataset)
print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(loader_val.dataset),
    100. * correct / len(loader_val.dataset)))